# 実験を途中保存する

現在の実験パッケージの内容をGIN-forkに途中保存します。  
上から順番に全て実行してください。  

※ 同期できるのは、実験実行環境に紐づいた実験名の実験パッケージの内容のみです。他の実験パッケージの変更内容は同期されません。  

実験フロートップページに戻る場合は[こちら](../experiment.ipynb)。新規タブで開きます。  

## 0. 研究リポジトリ名・実験パッケージ名を確認する  
以下のセルを実行すると、この実験実行環境で操作する実験パッケージの名前と、実験パッケージの存在する研究リポジトリ名を確認できます。  
**このタスクで同期されるのは、表示された名前の実験パッケージ内のデータのみです。**  


In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils

%store -r
if 'EXPERIMENT_TITLE' not in locals().keys() : EXPERIMENT_TITLE = '-'
utils.show_name('blue', EXPERIMENT_TITLE)

## 1. コミットメッセージを変数に入力する

以下のセルを実行して、実験作業のログとして残す短いメッセージを入力してください。  
※ 入力値に誤りがある場合、次に進む前に、もう一度実行することで訂正ができます。

In [ ]:
from IPython.display import clear_output
print('入力完了後、Enterキーを押下してください。')
message = input()
clear_output()
print('コミットメッセージ：', message)

## 2. 実験内容をGIN-forkに途中保存する
現在の実験パッケージの内容をGIN-forkへと同期します。  
以下のセルを順に実行してください。


In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os
import glob


%store -r EXPERIMENT_TITLE
experiment_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE
os.chdir(experiment_path)

#**************************************************#
#* Generate a list of folder paths to be managed by Git-annex. #
#**************************************************#
dirlist=[]
filelist=[]
annexed_save_path=[]

# Recursively search under the experimental package to obtain a list of absolute directory paths.
for root, dirs, files in os.walk(top=experiment_path):
    for dir in dirs:
        dirPath = os.path.join(root, dir)
        dirlist.append( dirPath )   


# Add directory paths containing the string "output_data" that are not included under input_data to annexed_save_path.
output_data_path = [ s for s in dirlist if 'output_data' in s ]
for output_data in output_data_path:  
    if  "input_data" not in output_data:
        annexed_save_path.append( output_data )

# Add the input_data directory to annexed_save_path.
annexed_save_path.append( experiment_path + '/input_data'  )

# Generate a list of file paths to which metadata is to be assigned.
gitannex_files = []
for path in annexed_save_path:
    gitannex_files += [p for p in glob.glob(path+'/**', recursive=True)
             if os.path.isfile(p)]

#********************************************************#
#* Generate a list of directory paths and file paths to be managed by Git. #
#********************************************************#
# Obtain a list of directories and files directly under the experimental package.
files = os.listdir()

# Delete Git-annex managed directories (input_data and output_data) from the retrieved list.
dirs = [f for f in files if os.path.isdir(f)]

for dirname in dirs:
    if dirname == 'input_data' :
        dirs.remove('input_data')

    if dirname == 'output_data' :
        dirs.remove('output_data')

for dirname in dirs:
    if dirname != 'ci' and dirname != 'source':
        full_param_dir = '{}/{}/params'.format(experiment_path,dirname)
        if os.path.isdir(full_param_dir):
            dirs.remove(dirname)
            ex_param_path = '{}/{}'.format(experiment_path, dirname)
            ex_param_path_childs = os.listdir(ex_param_path)
            for ex_param_path_child in ex_param_path_childs:
                if ex_param_path_child != 'output_data':
                    dirs.append('{}/{}'.format(dirname,ex_param_path_child))

# Obtain files directly under the experimental package.
files = [f for f in files if os.path.isfile(f)]

# Generate a list of folder paths and file paths to be managed by Git.
files.extend(dirs)
save_path = []
for file in files:
    save_path.append(experiment_path + '/' + file)

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[GINへの同期の失敗を解消する。](../conflict_helper.ipynb)を参照して、競合を解消してください。

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])

is_ok = utils.syncs_with_repo(git_path=save_path, gitannex_path=annexed_save_path, gitannex_files=gitannex_files, message=EXPERIMENT_TITLE + '_' + message)

## 3. 実験フロートップページに遷移する

以下のセルを実行し、表示されるリンクをクリックして実験フロートップページに戻ってください。  

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='../experiment.ipynb'>実験フロートップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))